<a href="https://colab.research.google.com/github/nawnie/EveryDream2trainer/blob/main/Train_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

EveryDream2 Colab Edition

In [ ]:
#@title # Install python 3.10 
import os
from IPython.display import clear_output
!wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
!bash ./py310.sh -b -f -p /usr/local
!python -m ipykernel install --name "py310" --user
clear_output()
os.kill(os.getpid(), 9)

In [ ]:
#@title verify python version
!python --version

In [ ]:
#@title Optional connect Gdrive
from google.colab import drive
drive.mount('/content/drive')


!mkdir /content/drive/MyDrive/logs/ckpt

In [ ]:
#@markdown # Install Dependencies
from IPython.display import clear_output
from subprocess import getoutput
s = getoutput('nvidia-smi')
!pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 --extra-index-url "https://download.pytorch.org/whl/cu116"
!pip install transformers==4.25.1
!pip install diffusers[torch]==0.10.2
!pip install pynvml==11.4.1
!pip install bitsandbytes==0.35.0
!pip install ftfy==6.1.1
!pip install aiohttp==3.8.3
!pip install tensorboard>=2.11.0
!pip install protobuf==3.20.1
!pip install wandb==0.13.6
!pip install pyre-extensions==0.0.23
if "A100" in s:
  !pip install https://huggingface.co/industriaditat/xformers_precompiles/blob/main/A100_13dev/xformers-0.0.13.dev0-py3-none-any.whl
else:
  !pip install https://huggingface.co/industriaditat/xformers_precompiles/resolve/main/T4_13dev/xformers-0.0.13.dev0-py3-none-any.whl
!pip install pytorch-lightning==1.6.5
!pip install OmegaConf==2.2.3
!pip install numpy==1.23.5
!pip install colorama
!pip install keyboard
clear_output()
!git clone https://github.com/victorchall/EveryDream2trainer.git
%cd /content/EveryDream2trainer
!wget "https://raw.githubusercontent.com/nawnie/EveryDream2trainer/main/train_colab.py"
clear_output()
print("DONE!")

In [ ]:
#@title Get A Base Model
#@markdown in this space you can type any url however i provide 2 options in the drop down
from IPython.display import clear_output
!mkdir input
%cd /content/EveryDream2trainer
!python utils/get_yamls.py
!rm train.py
!wget "https://github.com/nawnie/EveryDream2trainer/blob/e605cc59fb1f84f4f8e9290d88f7ede7b4c4fe0c/train_colab.py"
!mkdir Ckpt_cache
MODEL_URL = "https://huggingface.co/panopstor/EveryDream/resolve/main/sd_v1-5_vae.ckpt" #@param ["https://huggingface.co/panopstor/EveryDream/resolve/main/sd_v1-5_vae.ckpt", "https://huggingface.co/hakurei/waifu-diffusion-v1-3/resolve/main/wd-v1-3-float16.ckpt"] {allow-input: true}
print("Downloading ")
!wget $MODEL_URL

%cd /content/EveryDream2trainer


clear_output()
print("DONE!")

In order to train, you need a base model on which to train. This is a one-time setup to configure base models when you want to use a particular base.


In [ ]:
import os
#@title Convert to diffuers currently only 1.5 based models
#@markdown * This Path should be to an ckpt you want to use the one we just downloaded will be inside EveryDream2trainer

Base_path = "/content/EveryDream2trainer/sd_v1-5_vae.ckpt" #@param {type:"string"}
#@markdown * the name that you will use when selecting this model in the future traning sessons.
save_name = "SD_1.5" #@param{type:"string"}

!python utils/convert_original_stable_diffusion_to_diffusers.py --scheduler_type ddim  \
--original_config_file v1-inference.yaml \
--image_size 512 \
--checkpoint_path "$Base_path" \
--prediction_type epsilon \
--upcast_attn False \
--dump_path "$save_name"


In [ ]:
#@title Resume from a diffuser model saved to your Gdrive
#@markdown * if you have preveiously saved diffusers on your drive you can slect them here
Resume_Model = "" #@param{type:"string"} 
save_name= Resume_Model

For a more indepth Explanation of each of these paramaters check out /content/EveryDream2trainer/doc.


After youve tried a few models you will find /content/EveryDream2trainer/doc/ATWEAKING.md to be extremly helpful.

In [ ]:
#@title 
#@markdown # Run Everydream 2

Save_to_Gdrive = True #@param{type:"boolean"}
#@markdown * Use resume to contnue training you just ran
resume = False #@param{type:"boolean"}
#@markdown * Checkpointing Saves Vram to allow larger batch sizes minor slow down on a single batch size but will can allow room for a higher traning resolution
Gradient_checkpointing = True #@param{type:"boolean"}
#@markdown * Xformers saves ram and offers a great speed up
Disable_Xformers = False #@param{type:"boolean"}
#@markdown * best to just read this if interested in shufflng tags /content/EveryDream2trainer/doc/SHUFFLING_TAGS.md
shuffle_tags = False #@param{type:"boolean"}
#@markdown * you can stop the text encoder to attempt to reduce overfitting when resuming an unfinished model
Disable_text_Encoder= False #@param{type:"boolean"}
Project_Name = "" #@param{type: 'string'}
Max_Epochs = 100 #@param {type:"slider", min:0, max:200, step:10}

#@markdown * The learning rate affects how much "training" is done on the model per training step. It is a very careful balance to select a value that will learn your data. See Advanced Tweaking for more info. Once you have started, the learning rate is a good first knob to turn as you move into more advanced tweaking.

Learning_Rate = 1e-6 #@param{type: 'number'}

#@markdown * A learning rate scheduler can change your learning rate as training progresses.

#@markdown * The constant scheduler is the default and keeps your LR set to the value you set in the command line. That's really it for constant! I recommend sticking with it until you are comfortable with general training.

Schedule = "polynomial" #@param ["constant", "polynomial", "linear", "cosine"] {allow-input: true}
Resolution = 512#@param{type:"integer"} 

#@markdown * Batch size is also another "hyperparamter" of itself and there are tradeoffs. It may not always be best to use the highest batch size possible. Once of the primary reasons to change it is if you get "CUDA out of memory" errors where lowering the value may help.

#@markdown * While very small batch sizes can impact performance negatively, at some point larger sizes have little impact on overall speed as well, so shooting for the moon is not always advisable. Changing batch size may also impact what learning rate you use, with typically larger batch_size requiring a slightly higher learning rate.

Batch_Size = 4 #@param{type: 'number'}

#@markdown * Gradient accumulation is sort of like a virtual batch size increase use this to increase batch size with out increasing vram usage
#@markdown * 1 or 5 steps will take the same vram as a batch of 1
#@markdown * in colab free teir you can expect the fastest proformance from a batch of 4 and a step of 2 giving us a total batch size of 8 at 512 resolution 
#@markdown * Due to bucketng you may need to decresse batch size to 3
#@markdown * Remember fast Doesnt always mean better nor does slow experement

Gradient_steps = 2 #@param{type: 'number'}
Dataset_Location = "/content/drive/MyDrive/Zmanda" #@param {type:"string"}
dataset = Dataset_Location
model = save_name

Save_every_N_epoch = 25#@param{type:"integer"}

#@markdown You can set your own sample prompts by adding them, one line at a time, to sample_prompts.txt.

Steps_between_samples = 30#@param{type:"integer"}

Drive=""
if Save_to_Gdrive:
  Drive = "--logdir /content/drive/MyDrive/logs --save_ckpt_dir /content/drive/MyDrive/logs/ckpt"

if Max_Epochs==0:
  Max_Epoch=1

if resume:
  model = "findlast"


Gradient = ""
if Gradient_checkpointing:
  Gradient = "--gradient_checkpointing "
if "A100" in s:
  Gradient = ""

DX = ""  
if Disable_Xformers:
  DX = "--disable_xformers "

shuffle = ""
if shuffle_tags:
  shuffle = "--shuffle_tags "

textencode = ""
if Disable_text_Encoder:
  textencode = "--disable_textenc_training Train_text "


!python train_colab.py --resume_ckpt "$model" \
  $textencode \
  $Gradient \
  $shuffle \
  $Drive \
  $DX \
  --amp \
  --batch_size $Batch_Size \
  --grad_accum 2 \
  --cond_dropout 0.00 \
  --data_root "$dataset" \
  --flip_p 0.01 \
  --lr $Learning_Rate \
  --lr_decay_steps 0 \
  --lr_scheduler "$Schedule" \
  --lr_warmup_steps 0 \
  --max_epochs $Max_Epochs \
  --project_name "$Project_Name" \
  --resolution $Resolution \
  --sample_prompts "sample_prompts.txt" \
  --sample_steps $Steps_between_samples \
  --save_every_n_epoch $Save_every_N_epoch \
  --seed 555 \
  --shuffle_tags \
  --useadam8bit \
  --notebook


In [ ]:
#@title Adv Setup use the .Json to setup your paramaters
!python train.py --config train.json